In [ ]:
!nvidia-smi
!pip install -U ultralytics --no-deps


In [ ]:
from ultralytics import YOLO
import os, yaml, shutil, random, time, threading
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
DATASET_PATH = '/kaggle/input/microscopy-cell-segmentation-yolov12'
WORK_DIR = '/kaggle/working'

train_img = f'{DATASET_PATH}/train/images'
train_lbl = f'{DATASET_PATH}/train/labels'
val_img   = f'{DATASET_PATH}/valid/images'
val_lbl   = f'{DATASET_PATH}/valid/labels'
test_img  = f'{DATASET_PATH}/test/images'
test_lbl  = f'{DATASET_PATH}/test/labels'


In [ ]:
#LOAD CLASS INFO


with open(f'{DATASET_PATH}/data.yaml') as f:
    data_yaml = yaml.safe_load(f)

CLASS_NAMES = data_yaml['names']
NC = data_yaml['nc']


In [ ]:
#CREATE SSL SPLITS (80/10/10 + 20/80)



all_train = [f for f in os.listdir(train_img) if f.endswith(('.jpg','.png'))]

labeled, unlabeled = train_test_split(
    all_train, test_size=0.8, random_state=42
)

splits = ['labeled','unlabeled','train','val','test']
for s in splits:
    os.makedirs(f'{WORK_DIR}/images/{s}', exist_ok=True)
    os.makedirs(f'{WORK_DIR}/labels/{s}', exist_ok=True)


In [ ]:
#COPY DATA


def copy_pair(img, src_img, src_lbl, dst_img, dst_lbl):
    shutil.copy(f'{src_img}/{img}', f'{dst_img}/{img}')
    lbl = img.replace('.jpg','.txt').replace('.png','.txt')
    if os.path.exists(f'{src_lbl}/{lbl}'):
        shutil.copy(f'{src_lbl}/{lbl}', f'{dst_lbl}/{lbl}')

# Labeled
for img in labeled:
    copy_pair(img, train_img, train_lbl,
              f'{WORK_DIR}/images/labeled',
              f'{WORK_DIR}/labels/labeled')

# Unlabeled (images only)
for img in unlabeled:
    shutil.copy(f'{train_img}/{img}', f'{WORK_DIR}/images/unlabeled/{img}')

# Full train
for img in all_train:
    copy_pair(img, train_img, train_lbl,
              f'{WORK_DIR}/images/train',
              f'{WORK_DIR}/labels/train')

# Validation
for img in os.listdir(val_img):
    copy_pair(img, val_img, val_lbl,
              f'{WORK_DIR}/images/val',
              f'{WORK_DIR}/labels/val')

# Test
for img in os.listdir(test_img):
    copy_pair(img, test_img, test_lbl,
              f'{WORK_DIR}/images/test',
              f'{WORK_DIR}/labels/test')


In [ ]:
#YAML FILES

data_labeled = {
    'path': WORK_DIR,
    'train': 'images/labeled',
    'val': 'images/val',
    'test': 'images/test',
    'nc': NC,
    'names': CLASS_NAMES
}

data_combined = data_labeled.copy()
data_combined['train'] = 'images/train'

with open(f'{WORK_DIR}/data_labeled.yaml','w') as f:
    yaml.dump(data_labeled,f)

with open(f'{WORK_DIR}/data_combined.yaml','w') as f:
    yaml.dump(data_combined,f)


In [ ]:
#KEEP SESSION ALIVE

def keep_alive():
    while True:
        time.sleep(60)
        print("⏳ Session alive")

threading.Thread(target=keep_alive, daemon=True).start()


In [ ]:
#BASELINE TRAINING (20% LABELED)

baseline = YOLO('yolo12s-seg.yaml').load('yolo12s.pt')

baseline.train(
    data=f'{WORK_DIR}/data_labeled.yaml',
    epochs=50,
    imgsz=640,
    batch=64,
    device='0,1',
    project='runs',
    name='baseline',
    exist_ok=True
)


In [ ]:
#SSL #1: PSEUDO-LABELING

pseudo_lbl_dir = f'{WORK_DIR}/labels/pseudo'
os.makedirs(pseudo_lbl_dir, exist_ok=True)

unlabeled_img_dir = f'{WORK_DIR}/images/unlabeled'

for img in os.listdir(unlabeled_img_dir):
    if not img.endswith(('.jpg', '.png')):
        continue

    img_path = f'{unlabeled_img_dir}/{img}'
    results = baseline.predict(img_path, conf=0.7, verbose=False)

    if results and results[0].masks is not None:
        # Create label filename explicitly
        label_name = img.replace('.jpg', '.txt').replace('.png', '.txt')
        label_path = f'{pseudo_lbl_dir}/{label_name}'

        # Save pseudo-labels to FILE (not directory)
        results[0].save_txt(label_path)

print("✅ Pseudo-label generation completed.")

# Merge pseudo-labels into training labels
shutil.copytree(
    pseudo_lbl_dir,
    f'{WORK_DIR}/labels/train',
    dirs_exist_ok=True
)

print("✅ Pseudo-labels merged with training labels.")

# Train pseudo-label model
pseudo = YOLO('yolo12s-seg.yaml').load('yolo12s.pt')

pseudo.train(
    data=f'{WORK_DIR}/data_combined.yaml',
    epochs=15,        # increase later
    imgsz=416,
    batch=4,
    device='0,1',        # USE SINGLE GPU FOR STABILITY
    project='runs',
    name='pseudo_label',
    exist_ok=True
)


In [ ]:
#SSL #2: MEAN TEACHER (EMA-Style)

mean_teacher = YOLO('yolo12n-seg.yaml').load('yolo12n.pt')

mean_teacher.train(
    data=f'{WORK_DIR}/data_combined.yaml',
    epochs=15,
    imgsz=416,
    batch=4,
    augment=True,
    device='0,1',
    project='runs',
    name='mean_teacher',
    exist_ok=True
)


In [ ]:
#SSL #3: FIXMATCH (Strong Aug)

fixmatch = YOLO('yolo12s-seg.yaml').load('yolo12s.pt')

fixmatch.train(
    data=f'{WORK_DIR}/data_combined.yaml',
    epochs=50,
    imgsz=640,
    batch=64,
    augment=True,
    mosaic=1.0,
    mixup=0.2,
    device='0,1',
    project='runs',
    name='fixmatch',
    exist_ok=True
)


In [ ]:
#EVALUATION (MASK mAP) [FIXED FOR YOLOv12]

baseline_metrics = baseline.val(data=f'{WORK_DIR}/data_labeled.yaml')
pseudo_metrics   = pseudo.val(data=f'{WORK_DIR}/data_combined.yaml')
mt_metrics       = mean_teacher.val(data=f'{WORK_DIR}/data_combined.yaml')
fix_metrics      = fixmatch.val(data=f'{WORK_DIR}/data_combined.yaml')

metrics = {
    "Baseline": baseline_metrics.seg.map,
    "PseudoLabel": pseudo_metrics.seg.map,
    "MeanTeacher": mt_metrics.seg.map,
    "FixMatch": fix_metrics.seg.map
}

import pandas as pd
pd.DataFrame.from_dict(
    metrics,
    orient='index',
    columns=['Mask mAP@0.5:0.95']
)


In [ ]:
metrics_50 = {
    "Baseline": baseline_metrics.seg.map50,
    "PseudoLabel": pseudo_metrics.seg.map50,
    "MeanTeacher": mt_metrics.seg.map50,
    "FixMatch": fix_metrics.seg.map50
}

pd.DataFrame.from_dict(
    metrics_50,
    orient='index',
    columns=['Mask mAP@0.5']
)


In [ ]:
#VISUALIZATION

samples = random.sample(os.listdir(f'{WORK_DIR}/images/test'), 3)

models = [baseline, pseudo, mean_teacher, fixmatch]
titles = ['Baseline','Pseudo','MeanTeacher','FixMatch']

for img in samples:
    path = f'{WORK_DIR}/images/test/{img}'
    img0 = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

    fig, ax = plt.subplots(1,5,figsize=(25,5))
    ax[0].imshow(img0); ax[0].set_title("Input"); ax[0].axis('off')

    for i,m in enumerate(models):
        r = m.predict(path, conf=0.25)[0]
        ax[i+1].imshow(cv2.cvtColor(r.plot(), cv2.COLOR_BGR2RGB))
        ax[i+1].set_title(titles[i])
        ax[i+1].axis('off')

    plt.show()


In [ ]:
#TRAINING GRAPHS (LOSS & METRIC CURVES)

from IPython.display import Image, display

experiments = [
    'baseline',
    'pseudo_label',
    'mean_teacher',
    'fixmatch'
]

for exp in experiments:
    path = f'runs/{exp}/results.png'
    if os.path.exists(path):
        print(f'📈 Training curves: {exp}')
        display(Image(filename=path))
    else:
        print(f'⚠️ Missing results for {exp}')


In [ ]:
#SEGMENTATION MASK VISUALIZATIONS 

import random
import cv2
import matplotlib.pyplot as plt

test_images = os.listdir(f'{WORK_DIR}/images/test')
samples = random.sample(test_images, 3)

models = [baseline, pseudo, mean_teacher, fixmatch]
titles = ['Baseline', 'Pseudo-Label', 'Mean Teacher', 'FixMatch']

for img in samples:
    img_path = f'{WORK_DIR}/images/test/{img}'
    image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

    fig, axes = plt.subplots(1, 5, figsize=(25, 5))

    axes[0].imshow(image)
    axes[0].set_title('Input Image')
    axes[0].axis('off')

    for i, model in enumerate(models):
        r = model.predict(img_path, conf=0.25, imgsz=416, verbose=False)[0]
        axes[i+1].imshow(cv2.cvtColor(r.plot(), cv2.COLOR_BGR2RGB))
        axes[i+1].set_title(titles[i])
        axes[i+1].axis('off')

    plt.tight_layout()
    plt.show()
